# 1. Load the bulk .npy files and concatenate into single array 

In [1]:
from sklearn.model_selection import train_test_split
# For TF MONO Ratio Only (enhanced)

import numpy as np

#a = np.load("Modified_EMO_6_FULL_VALUES.npy")
a = np.load("Modified_EMO_4_FULL_VALUES.npy")

X = a[:-1]
X.shape
X[0][0]

0.0

# 2. Load the class label file class_label.txt

In [2]:
# 'reuters21578 _enc.txt' class label or y of 7125 rows 
y=np.genfromtxt('class_label.txt', delimiter=',', dtype='int32', skip_header=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



# 3. Load WebKB_version2.csv to obtain Features

In [3]:
import numpy as np

import pandas as pd
data=pd.read_csv('WebKB_version2.csv')

import nltk
stopwords = nltk.corpus.stopwords.words('english')

import string
import re

from nltk.stem import PorterStemmer
ps = PorterStemmer()

from math import *

# 4. Pre-processing

In [4]:
#Pre-processing
def clean_text(txt):
    txt = "".join([c for c in txt if c not in string.punctuation])
    tokens = re.split('\W+',txt)
    txt = [ps.stem(word) for word in tokens if word not in stopwords]
    return txt

# 5. Generating Features

In [5]:
# Only simulate min_df = 0 for extracting 12418  features

from sklearn.feature_extraction.text import CountVectorizer

cv_vect = CountVectorizer(analyzer = clean_text, lowercase=True)
#cv_vect = CountVectorizer(analyzer='word', encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words='english', token_pattern='(?u)\b\w\w+\b', max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False )

XCV = cv_vect.fit(data['document'])
X_CV = cv_vect.transform(data['document'])
X_CV.shape

(8324, 7708)

In [6]:
features = cv_vect.get_feature_names() #for 12418 

In [7]:
# tokenizing each row and transfering to row list
row = data['document'].apply(lambda x: clean_text(x)) 
#row = cv_vect.inverse_transform(X_CV)

In [8]:
# chi Square 

from sklearn.feature_selection import chi2
f_p_values = chi2(X_train,y_train)

p_values = pd.Series(f_p_values[0])
p_values.index = features
sorted_features = p_values.sort_values(ascending = True)

In [9]:
#f_p_values[0]

In [10]:
#f_p_values[1]

#  Modify the Number of Features

In [11]:
# Modify values for number of Features =  500, 1000, 2000, 3000, 4000, 5000 ... 10000

num_features = 500

In [12]:
selected_features=sorted_features[0:num_features]

In [13]:
selected_features.index

Index(['cecilspec1', 'refman', 'latest', 'g221', 'vortex', 'bench', 'vortex1',
       'languagepeopl', '2110', 'loadbalanc',
       ...
       'ctc', 'psfile', 'jpubl', 'robotgroup', 'robotcatchabst', 'jtoverview',
       'jtsyllabusmw', 'jtsyllabusmwf', 'intercom', 'cs418sp94'],
      dtype='object', length=500)

In [14]:
# For TF MONO Ratio Only:
##################PostGlobalMONO = np.genfromtxt('GlobalMONO_Ratio_12418.txt', delimiter=',', dtype='float16') #for 12418 features

# For TF MONO  Only:
#PostGlobalMONO = np.genfromtxt('GlobalMONO_12418.txt', delimiter=',', dtype='float16') #for 12418 features

# For TF MONO Ratio Distance H2 Only:
#PostGlobalMONO = np.genfromtxt('Global_MONO_Distance_H2.txt', delimiter=',', dtype='float16') #for 12418 features

# H3 ------ For TF MONO Ratio Distance H3 Only:
#PostGlobalMONO = np.genfromtxt('Global_MONO_Distance_H3.txt', delimiter=',', dtype='float16') #for 12418 features

# H4 ------ For TF MONO Ratio Distance H4 Only:
#PostGlobalMONO = np.genfromtxt('Global_MONO_Distance_H4.txt', delimiter=',', dtype='float16') #for 12418 features



#PostGlobalMONO = np.genfromtxt('Global_MONO_H1.txt', delimiter=',', dtype='float16') 

#EMO = 2
PostGlobalMONO = np.genfromtxt('Modified_EMO_2.txt', delimiter=',', dtype='float16') 


PostGlobalMONO.shape

(7708,)

In [15]:
PostGlobalMONO = PostGlobalMONO.astype("float16")

In [16]:
# Creating a Data Frame for Global Mono from generated computing in excel containing 2 columns = features and 1 + GlobalMONO * 6.0
df_PostGlobalMONO = pd.DataFrame(data={'Features': features, 'Global_Mono': PostGlobalMONO, 'p_values': p_values})

In [17]:
df_PostGlobalMONO

,Features,Global_Mono,p_values
,,1.000000,0.149626
0,0,2.541016,4.111511
0002,0002,1.006836,0.197850
001,001,1.057617,1.384979
002,002,1.000977,0.066484
...,...,...,...
zou,zou,1.000000,0.305628
zpl,zpl,1.000000,0.553498
zuelzk,zuelzk,1.000000,NaN
zymurgi,zymurgi,1.000000,0.099726


In [18]:
p_values

           0.149626
0          4.111511
0002       0.197850
001        1.384979
002        0.066484
             ...   
zou        0.305628
zpl        0.553498
zuelzk          NaN
zymurgi    0.099726
zzhen00    0.361174
Length: 7708, dtype: float64

In [19]:
# displaying created dataframe
filtered_df_PostGlobalMONO = df_PostGlobalMONO.sort_values(by = ['p_values'], ascending = False)
res_df_PostGlobalMONO=filtered_df_PostGlobalMONO.head(num_features)
res_df_PostGlobalMONO

,Features,Global_Mono,p_values
project,project,6.804688,2319.523283
depart,depart,6.976562,492.564761
staff,staff,6.976562,477.609906
faculti,faculti,1.000000,406.595961
student,student,6.925781,366.664774
...,...,...,...
kmedwww,kmedwww,1.000000,1.426347
css,css,1.006836,1.426347
galileo,galileo,1.000000,1.426347
prolang,prolang,1.000000,1.426347


In [20]:
# Chi2 max Features 500, 1000, 2000, 3000, 4000, 5000 ... 10000
chi2max_df_PostGlobalMONO = filtered_df_PostGlobalMONO['Global_Mono'][0:num_features]
chi2max_df_PostGlobalMONO

project    6.804688
depart     6.976562
staff      6.976562
faculti    1.000000
student    6.925781
             ...   
kmedwww    1.000000
css        1.006836
galileo    1.000000
prolang    1.000000
muster     1.000000
Name: Global_Mono, Length: 500, dtype: float16

In [21]:
chi2max_df_PostGlobalMONO.index

Index(['project', 'depart', 'staff', 'faculti', 'student', 'cours', 'http',
       'html', 'edu', 'misc',
       ...
       'gulf', 'gu', 'hpcd', 'cord', 'vivaldi', 'kmedwww', 'css', 'galileo',
       'prolang', 'muster'],
      dtype='object', length=500)

In [22]:
from math import *

In [23]:
temp = np.zeros((1,num_features), dtype="float16")
TF_MONO_TABLE = pd.DataFrame(np.zeros((1,num_features )))

#limit = 8324
# 6 minutes is 1k rows for 12418 features
# 2 minutes is 1k rows for 500 features
for i in range(0,8323):
    for j in row[i]:
        
        tf = row[i].count(j) / len(row[i])
        #print(tf)
       
        #print("tf OF {} = {} / {} = {}".format(j,data['document'][i].count(j),len(row[i]),tf))
        
    
        
        index_of_term = np.where(res_df_PostGlobalMONO == j)
        actual_index = index_of_term[0]
        
        # MONOfromDataFrame Represent the MONO in every row / document:
        MONOfromDataFrame = res_df_PostGlobalMONO['Global_Mono'].iloc[actual_index]
      
        ###print("Actual index is {}".format(actual_index))
        
        
        
         # A. TF MONO:
        TF_MONO_result = tf * MONOfromDataFrame
        
        # B. Squared Root TF MONO:
        #TF_MONO_result = sqrt(tf) * MONOfromDataFrame
        
        ###print("[Row {}] TF OF {} = {} / {} = {}".format(i+1,j,row[i].count(j),len(row[i]),tf))
        ###print("* Global MONO of ")
        ###print(res_df_PostGlobalMONO['Features'].iloc[actual_index])
        ###print(MONOfromDataFrame)
        ###print("The result is {}".format(TF_MONO_result))
        ###print("------------------------------------------------------------")
        #print(j,tf, df_PostGlobalMONO['Global_Mono'].iloc[actual_index], TF_MONO_result)
        
        temp[0][actual_index] = TF_MONO_result
        
        # Coverting Temp Array into Data Frame
#Modify here 6:
    First_Result = pd.DataFrame(temp, columns = chi2max_df_PostGlobalMONO.index)
    First_Result = First_Result.astype("float16")
    temp = np.zeros((1,num_features), dtype="float16")
    
    TF_MONO_TABLE = np.append(TF_MONO_TABLE, np.array(First_Result), axis=0)
    
    First_Result.drop(First_Result.index[:], inplace=True)
# appending another dataframe as another row

#TF_MONO_TABLE2 = pd.DataFrame(np.ones((1,29523)))

#Modify here 7:

#TF_MONO_TABLE = pd.DataFrame(Second_Result, columns = features)   
TF_MONO_TABLE.shape

(8324, 500)

# Export result as numpy

In [25]:
TF_MONO_TABLE = TF_MONO_TABLE.astype("float16")

np.save("TF_Modified_EMO_4_EMO_2_500_Features.npy",TF_MONO_TABLE)

In [26]:
#pd.DataFrame(TF_MONO_TABLE, columns = chi2max_df_PostGlobalMONO.index)  

# Clear numpy array / Data Frame

In [27]:

TF_MONO_TABLE = np.array([])

#clear the array
temp = np.zeros((1,num_features), dtype="float16")
temp.shape

(1, 500)